In [6]:
import requests
import random
from typing import Dict, Any, Optional


BASE_URL = "https://berghain.challenges.listenlabs.ai/"  # ← replace with the real host
PLAYER_ID = "5d7389b8-683e-453b-a35f-6d21b1b5e94e"
SCENARIO = 1  # choose 1, 2, or 3

def create_new_game(scenario: int, player_id: str) -> Dict[str, Any]:
    url = f"{BASE_URL}/new-game"
    res = requests.get(url, params={"scenario": scenario, "playerId": player_id}, timeout=30)
    res.raise_for_status()
    data = res.json()
    print("New game:", data["gameId"])
    return data


def decide_and_next(game_id: str, person_index: int, accept: Optional[bool] = None) -> Dict[str, Any]:
    url = f"{BASE_URL}/decide-and-next"
    params = {"gameId": game_id, "personIndex": person_index}
    if accept is not None:
        params["accept"] = str(accept).lower()  # true/false in query string
    res = requests.get(url, params=params, timeout=30)
    res.raise_for_status()
    return res.json()

new_game = create_new_game(SCENARIO, PLAYER_ID)
game_id = new_game["gameId"]
constraints = new_game.get("constraints", [])
stats = new_game.get("attributeStatistics", {})

print(constraints)
print(stats)

New game: 39a22d6e-5cd6-4049-80e9-c3043f299c5a
[{'attribute': 'young', 'minCount': 600}, {'attribute': 'well_dressed', 'minCount': 600}]
{'relativeFrequencies': {'well_dressed': 0.3225, 'young': 0.3225}, 'correlations': {'well_dressed': {'well_dressed': 1, 'young': 0.18304299322062992}, 'young': {'well_dressed': 0.18304299322062992, 'young': 1}}}


In [ ]:
state = decide_and_next(game_id, person_index=0)
if state.get("status") == "failed":
    raise RuntimeError(f"Game failed: {state.get('reason')}")
admitted = state.get("admittedCount", 0)
rejected = state.get("rejectedCount", 0)
next_person = state.get("nextPerson")
total_w = 0
total_y = 0
n_accepted = 0
while state.get("status") == "running" and next_person:
        idx = next_person["personIndex"]
        attrs = next_person["attributes"]  # { attributeId: bool }
        w, y = attrs.get("well_dressed", False), attrs.get("young", False)
        #accept = decide(y, w)
        R = 1000 - (total_y + total_w)
        if total_y < 600 and total_w < 600:
            if y and w:
                accept = True
                total_w += 1
                total_y += 1
            elif y:
                if (R - 1) >= max(max(0, 600 - total_y + 1), max(0, 600 - total_w)):
                    accept = True
                    total_y += 1
                else:
                    accept = False
            elif w:
                if (R - 1) >= max(max(0, 600 - total_y), max(0, 600 - total_w + 1)):
                    accept = True
                    total_w += 1
            else:
                s = R - max(max(0, 600 - total_y), max(0, 600 - total_w))
                if s > 0:
                    number = random.random()
                    if number <= min(0.0237, s/20):
                        accept = True
                else:
                    accept = False
        else:
            accept = True
        state = decide_and_next(game_id, person_index=idx, accept=accept)
        status = state.get("status")
        if status == "failed":
            raise RuntimeError(f"Game failed: {state.get('reason')}")

        admitted = state.get("admittedCount", admitted)
        rejected = state.get("rejectedCount", rejected)
        next_person = state.get("nextPerson")

        print(f"Person {idx}: accept={accept} | admitted={admitted} rejected={rejected}")

        # 5) done
if state.get("status") == "completed":
        print("Game completed.")
        print("Final rejected count:", state.get("rejectedCount"))
else:
        print("Game ended with state:", state)

New game: 7ec7dc2a-fb85-437d-ae8c-46a96a462c70
Person 0: accept=True | admitted=1 rejected=0
Person 1: accept=True | admitted=2 rejected=0
Person 2: accept=True | admitted=3 rejected=0
Person 3: accept=True | admitted=4 rejected=0
Person 4: accept=True | admitted=5 rejected=0
Person 5: accept=True | admitted=6 rejected=0
Person 6: accept=True | admitted=7 rejected=0
Person 7: accept=True | admitted=8 rejected=0
Person 8: accept=True | admitted=9 rejected=0
Person 9: accept=True | admitted=10 rejected=0
Person 10: accept=True | admitted=11 rejected=0
Person 11: accept=True | admitted=12 rejected=0
Person 12: accept=True | admitted=13 rejected=0
Person 13: accept=True | admitted=14 rejected=0
Person 14: accept=True | admitted=15 rejected=0
Person 15: accept=True | admitted=16 rejected=0
Person 16: accept=True | admitted=17 rejected=0
Person 17: accept=True | admitted=18 rejected=0
Person 18: accept=True | admitted=19 rejected=0
Person 19: accept=True | admitted=20 rejected=0
Person 20: a

RuntimeError: Game failed: Venue full but constraints not met: young: 302/600 (need 298 more), well_dressed: 304/600 (need 296 more)